This assignment was completed in collaboration with the Black Redwood Capstone Team. All rights reserved.

In [ ]:
import pandas as pd
from google.colab import files
files.upload()
files.upload()

order_products_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')

order_products_train.head(), products.head()




Saving order_products__train.csv to order_products__train (3).csv


Saving products.csv to products (2).csv


(   order_id  product_id  add_to_cart_order  reordered
 0         1       49302                  1          1
 1         1       11109                  2          1
 2         1       10246                  3          0
 3         1       49683                  4          0
 4         1       43633                  5          1,
    product_id                                       product_name  aisle_id  \
 0           1                         Chocolate Sandwich Cookies        61   
 1           2                                   All-Seasons Salt       104   
 2           3               Robust Golden Unsweetened Oolong Tea        94   
 3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
 4           5                          Green Chile Anytime Sauce         5   
 
    department_id  
 0             19  
 1             13  
 2              7  
 3              1  
 4             13  )

In [ ]:
merged_data = pd.merge(order_products_train, products, on='product_id')
merged_data.rename(columns={'order_id': 'customer_id',
                           'product_id': 'product_id',
                           'product_name': 'product_category',
                           'add_to_cart_order': 'purchase_amount'}, inplace=True)
merged_data.head()

,customer_id,product_id,purchase_amount,reordered,product_category,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


In [ ]:
from collections import defaultdict

def mapper(data):
  category_count = defaultdict(int)
  product_count = defaultdict(int)
  order_count = defaultdict(int)

  for index, row in data.iterrows():
    customer_id = row['customer_id']
    product_id = row['product_id']
    product_category = row['product_category']
    purchase_amount = row['purchase_amount']

    category_count[product_category] += purchase_amount
    product_count[product_id] += purchase_amount
    order_count[customer_id] += purchase_amount

  return category_count, product_count, order_count

In [ ]:
def reducer(mapped_data):
  final_category_count = defaultdict(int)
  final_product_count = defaultdict(int)
  final_order_count = defaultdict(int)

  for category_count, product_count, order_count in mapped_data:
    for category, count in category_count.items():
      final_category_count[category] += count
    for product, count in product_count.items():
      final_product_count[product] += count
    for order, count in order_count.items():
      final_order_count[order] += count

  return final_category_count, final_product_count, final_order_count



In [ ]:
def combiner(mapped_data):
    combined_category_count = defaultdict(int)
    combined_product_count = defaultdict(int)
    combined_order_count = defaultdict(int)

    for category_count, product_count, order_count in mapped_data:
        for category, count in category_count.items():
            combined_category_count[category] += count
        for product, count in product_count.items():
            combined_product_count[product] += count
        for order, count in order_count.items():
            combined_order_count[order] += count

    return combined_category_count, combined_product_count, combined_order_count

In [ ]:
def get_top_products(product_count, products_df, n=5):
  sorted_products = sorted(product_count.items(), key=lambda x: x[1], reverse=True)
  top_products = sorted_products[:n]
  top_named_products = [(product_id, products_df[products_df['product_id'] == product_id]['product_name'].values[0], count) for product_id, count in top_products]
  return top_named_products

In [ ]:
def calculate_average_products(order_count):
  total_orders = len(order_count)
  total_products = sum(order_count.values())
  return total_products / total_orders if total_orders else 0

In [ ]:
import numpy as np

chunks = np.array_split(merged_data, 10)

mapped_data = [mapper(chunk) for chunk in chunks]

combined_data = combiner(mapped_data)

final_category_count, final_product_count, final_order_count = reducer([combined_data])

top_5_products = get_top_products(final_product_count, products)

average_products_per_order = calculate_average_products(final_order_count)

print("Total number of products purchased for each product category: ", dict(final_category_count))
print("Top 5 products purchased: ", top_5_products)
print("Average number of products purchased per order: {:.2f} ".format(average_products_per_order))

Total number of products purchased for each product category:  {'Bulgarian Yogurt': 48, 'Organic 4% Milk Fat Whole Milk Cottage Cheese': 1094, 'Organic Celery Hearts': 10634, 'Cucumber Kirby': 17892, 'Lightly Smoked Sardines in Olive Oil': 184, 'Bag of Organic Bananas': 73739, 'Organic Hass Avocado': 50853, 'Organic Whole String Cheese': 17316, 'Grated Pecorino Romano Cheese': 367, 'Spring Water': 10070, 'Organic Half & Half': 16716, 'Super Greens Salad': 5546, 'Cage Free Extra Large Grade AA Eggs': 266, 'Prosciutto, Americano': 1140, 'Organic Garnet Sweet Potato (Yam)': 24991, 'Asparagus': 36393, 'Shelled Pistachios': 728, 'Organic Biologique Limes': 974, 'Organic Raw Unfiltered Apple Cider Vinegar': 3915, 'Organic Baby Arugula': 26094, 'Organic Hot House Tomato': 3574, 'Green Peas': 4888, 'Bunched Cilantro': 17272, 'Flat Parsley, Bunch': 6249, 'Fresh Dill': 1139, 'Roasted Turkey': 2536, 'Organic Cucumber': 42951, 'Organic Grape Tomatoes': 36868, 'Organic Pomegranate Kernels': 481, 'O